In [40]:
import fitz
import PyPDF2

In [67]:
file_path = "/Users/liusean/Desktop/Comp Geo.pdf"


In [68]:
rgb_scale = 255
cmyk_scale = 100

#https://stackoverflow.com/questions/14088375/how-can-i-convert-rgb-to-cmyk-and-vice-versa-in-python

def rgb_to_cmyk(x):
    r, g, b = x[0], x[1], x[2]
    if (r == 0) and (g == 0) and (b == 0):
        # black
        return 0, 0, 0, cmyk_scale

    # rgb [0,255] -> cmy [0,1]
    c = 1 - r / float(rgb_scale)
    m = 1 - g / float(rgb_scale)
    y = 1 - b / float(rgb_scale)

    # extract out k [0,1]
    min_cmy = min(c, m, y)
    c = (c - min_cmy) 
    m = (m - min_cmy) 
    y = (y - min_cmy) 
    k = min_cmy

    # rescale to the range [0,cmyk_scale]
    return (round(c*cmyk_scale, 4), round(m*cmyk_scale, 4), round(y*cmyk_scale, 4), round(k*cmyk_scale, 4))

In [69]:
def get_color_counts(file_path):
    all_cmyk = {}
    all_rgb = {}
    doc = fitz.open(file_path)
    for page in doc:
        pix = page.get_pixmap()
        colors_rgb = pix.color_count(colors = True)
        colors_cmyk = {
            rgb_to_cmyk(x): colors_rgb[x] for x in colors_rgb
        }
        for cmyk, cnt in colors_cmyk.items():
            if cmyk not in all_cmyk:
                all_cmyk[cmyk] = 0
            all_cmyk[cmyk] += cnt
        for rgb, cnt in colors_rgb.items():
            if rgb not in all_rgb:
                all_rgb[rgb] = 0
            all_rgb[rgb] += cnt
    return all_cmyk, all_rgb
    

In [74]:
def get_page_dimensions(file_path):
    # returns width and height in mm

    page_dimensions = []

    pdf = PyPDF2.PdfReader(file_path, "rb")

    for p in pdf.pages:
        # w_in_user_space_units = p.mediaBox.getWidth()
        # h_in_user_space_units = p.mediaBox.getHeight()

        # 1 user space unit is 1/72 inch
        # 1/72 inch ~ 0.352 millimeters

        w = float(p.mediabox.width) * 0.352
        h = float(p.mediabox.height) * 0.352
        page_dimensions.append((w, h))
    return page_dimensions

In [76]:
%time
get_color_counts(file_path)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.15 µs


{(0.0, 19.6078, 95.2941, 1.9608): 8128,
 (0.0, 20.0, 95.6863, 1.5686): 39,
 (0.0, 14.902, 71.3725, 1.5686): 9,
 (0.0, 9.0196, 41.5686, 0.3922): 503,
 (0.0, 8.6275, 41.9608, 0.3922): 1162,
 (0.0, 9.4118, 41.5686, 0.3922): 116,
 (0.0, 8.6275, 41.9608, 0.7843): 110,
 (0.0, 10.5882, 50.5882, 1.5686): 9,
 (0.0, 12.9412, 62.7451, 1.1765): 8,
 (0.0, 14.1176, 62.3529, 0.3922): 10,
 (0.0, 10.5882, 49.8039, 0.7843): 6,
 (0.0, 7.0588, 41.1765, 1.5686): 33,
 (0.0, 8.6275, 41.5686, 0.3922): 2147,
 (0.0, 8.2353, 40.7843, 0.7843): 708,
 (0.0, 8.2353, 40.3922, 0.7843): 878,
 (0.0, 8.6275, 41.1765, 0.3922): 80035,
 (0.0, 8.6275, 40.7843, 0.3922): 1426,
 (0.0, 9.8039, 40.0, 0.0): 172,
 (0.0, 7.0588, 41.5686, 1.1765): 42,
 (0.0, 7.451, 41.1765, 1.1765): 192,
 (0.0, 8.6275, 40.3922, 0.3922): 351,
 (0.0, 9.4118, 41.1765, 0.0): 467,
 (0.0, 9.4118, 41.5686, 0.0): 480,
 (0.0, 9.4118, 41.9608, 0.0): 442,
 (0.0, 9.8039, 41.9608, 0.0): 108,
 (0.0, 9.4118, 40.7843, 0.0): 339,
 (0.0, 8.2353, 39.6078, 0.7843): 291,